In [1]:
import React from 'react';
import Highcharts from 'highcharts/highstock';
import HighchartsReact from 'highcharts-react-official';

const StockChart = ({ data }) => {
  const options = {
    title: {
      text: 'Stock Price'
    },
    xAxis: {
      type: 'datetime'
    },
    yAxis: {
      title: {
        text: 'Price'
      }
    },
    series: [{
      name: 'Stock Price',
      data: data,
      tooltip: {
        valueDecimals: 2
      }
    }]
  };

  return (
    <HighchartsReact
      highcharts={Highcharts}
      constructorType={'stockChart'}
      options={options}
    />
  );
};

export default StockChart;

SyntaxError: invalid syntax (2490503976.py, line 1)

In [ ]:
import React, { useState, useEffect, useCallback, useRef } from 'react';
import {
  Button,
  Text,
  Spinner,
  makeStyles,
  tokens,
} from "@fluentui/react-components";
import * as FluentIcons from "@fluentui/react-icons";
import SubgraphResults from './SubgraphResults';
import StockChart from './StockChart';

const useStyles = makeStyles({
  // ... (keep existing styles)
  chartContainer: {
    marginBottom: '20px',
    height: '400px',
  },
});

const EmailGenerator = () => {
  const styles = useStyles();
  const [chatMessages, setChatMessages] = useState([]);
  const [chatInput, setChatInput] = useState("");
  const [isChatActive, setIsChatActive] = useState(false);
  const [isFirstQuestion, setIsFirstQuestion] = useState(true);
  const [stockData, setStockData] = useState([]);

  // ... (keep other existing state variables and functions)

  useEffect(() => {
    // Hardcoded stock data
    const hardcodedStockData = [
      [Date.UTC(2023, 0, 1), 100],
      [Date.UTC(2023, 1, 1), 110],
      [Date.UTC(2023, 2, 1), 105],
      [Date.UTC(2023, 3, 1), 120],
      [Date.UTC(2023, 4, 1), 115],
      [Date.UTC(2023, 5, 1), 130],
      [Date.UTC(2023, 6, 1), 125],
      [Date.UTC(2023, 7, 1), 140],
      [Date.UTC(2023, 8, 1), 135],
      [Date.UTC(2023, 9, 1), 150],
      [Date.UTC(2023, 10, 1), 145],
      [Date.UTC(2023, 11, 1), 160],
    ];
    setStockData(hardcodedStockData);
  }, []);

  const handleChatSubmit = (e) => {
    e.preventDefault();
    if (chatInput.trim() && !isProcessing) {
      console.log("Submitting chat input:", chatInput);
      handleAction(chatInput);
      setChatInput("");
      if (isFirstQuestion) {
        setIsFirstQuestion(false);
      }
    }
  };

  if (isChatActive) {
    return (
      <div className={styles.chatPage}>
        <div className={styles.chatMessages}>
          {chatMessages.map((message, index) => {
            if (message.role === "subgraph") {
              const subgraphCount = Object.values(message.content).reduce((count, subgraphs) => count + Object.keys(subgraphs).length, 0);
              if (subgraphCount > 1) {
                return (
                  <div key={index} className={styles.subgraphMessage}>
                    <SubgraphResults results={message.content} />
                  </div>
                );
              } else {
                return null;
              }
            }
            return (
              <div
                key={index}
                className={`${styles.chatMessage} ${
                  message.role === "user" ? styles.userMessage : styles.assistantMessage
                }`}
              >
                {message.role === "user" ? (
                  message.content
                ) : (
                  <>
                    <div dangerouslySetInnerHTML={{ __html: message.content }} />
                    {index === chatMessages.length - 1 && message.role === "assistant" && (
                      <Button 
                        className={styles.exportButton}
                        onClick={handleExportToEmail}
                        icon={<FluentIcons.Mail24Regular />}
                      >
                        Export to Email
                      </Button>
                    )}
                  </>
                )}
              </div>
            );
          })}
          {!isFirstQuestion && (
            <div className={styles.chartContainer}>
              <StockChart data={stockData} />
            </div>
          )}
        </div>
        <form onSubmit={handleChatSubmit} className={styles.chatInput}>
          <input
            type="text"
            value={chatInput}
            onChange={(e) => setChatInput(e.target.value)}
            placeholder="Type your message..."
            className={styles.chatInputField}
            disabled={isProcessing}
          />
          <Button type="submit" disabled={isProcessing || !chatInput.trim()}>
            Send
          </Button>
        </form>
      </div>
    );
  }

  // ... (keep the rest of the component)
};

export default EmailGenerator;